In [ ]:
import json
import ndlib.models.ModelConfig as mc
import ndlib.models.opinions as op
import networkx as nx
from plots import *
from utils import *
import aggregateMetrics as am
import numpy as np

%matplotlib inline

# Single Execution

In [ ]:
graph = nx.complete_graph(100)

In [ ]:
#model parameter
mu = 0.5
epsilon = 0.3
gamma = 1.0
gamma_media = gamma
pm = 0.5
k = 3
media_op = [0.05, 0.5, 0.95]

#simulations parameter
max_iterations = 1000
sensibility = 0.00001
nsteady = 1000
nodeStatus = True
progressBar = True

In [ ]:
#create model
model = op.AlgorithmicBiasMediaModel(graph)

#create configuration
config = mc.Configuration()
config.add_model_parameter("mu", mu)
config.add_model_parameter("epsilon", epsilon)
config.add_model_parameter("gamma", gamma)
config.add_model_parameter("gamma_media", gamma)
config.add_model_parameter("p", pm)
config.add_model_parameter("k", k)

#configure model
model.set_initial_status(config)
model.set_media_opinions(media_op)

#perform iterations untill convergence
iterations = model.steady_state(max_iterations=max_iterations, nsteady=nsteady, sensibility=sensibility, node_status=nodeStatus, progress_bar=progressBar)

In [ ]:
plotevolution(iterations)
plotdistribution(list(iterations[-1]['status'].values()))

# Multiple Runs

In [ ]:
max_iterations = 100
nruns = 5

In [ ]:
k = 3
epsilons, gammas, pms, media_ops = [0.2, 0.3, 0.4, 0.5], [0.0, 0.5, 1.0, 1.5], [0.0, 0.1, 0.2, 0.3, 0.4, 0.5], [[0.0], [0.5], [0.05, 0.95], [0.05, 0.5, 0.95]]
graph = nx.complete_graph(100)

In [ ]:
for epsilon in epsilons:
    for gamma in gammas:
        for pm in pms:
            for media_op in media_ops:
                respath = f'../res/'
                name = f'e{epsilon}_g{gamma}_pm{pm}_mo{media_op}_maxit{max_iterations}'
                final_opinions, final_niter = read_dicts(respath, name)
                
                for run in range(nruns):
                    if str(run) in final_opinions.keys() and str(run) in final_niter.keys(): 
                        print('run already present. skipping.')
                        run += 1
                    else:
                        print(f'run {run} epsilon = {epsilon} gamma = {gamma} pm = {pm} media_op = {media_op}')    

                        #create model
                        model = op.AlgorithmicBiasMediaModel(graph)

                        #create configuration
                        config = mc.Configuration()
                        config.add_model_parameter("mu", 0.5)
                        config.add_model_parameter("epsilon", epsilon)
                        config.add_model_parameter("gamma", gamma)
                        config.add_model_parameter("gamma_media", gamma)
                        config.add_model_parameter("p", pm)
                        config.add_model_parameter("k", len(media_op))

                        #configure model
                        model.set_initial_status(config)
                        model.set_media_opinions(media_op)

                        #perform iterations untill convergence
                        iterations = model.steady_state(max_iterations=max_iterations, nsteady=1000, sensibility=0.00001, node_status=True, progress_bar=True)

                        finalopinions = iterations[-1]['status']
                        niter = int(iterations[-1]['iteration'])
                        
                        final_opinions[run] = finalopinions
                        final_niter[run] = niter
                        
                        write_dicts(respath, name, final_opinions, final_niter)

                        plotevolution(iterations, name=name, run=run)
                        plotdistribution(list(finalopinions.values()), name=name, run=run)


# Convergence Analysis

In [ ]:
def import_dicts(path, name):
    with open(f'{path}/final_opinions_{name}.json', 'rt') as file:
        final_opinions = json.load(file)
    with open(f'{path}/final_iterations_{name}.json', 'rt') as file:
        final_iterations = json.load(file)
    return final_opinions, final_iterations

In [ ]:
epsilons, gammas, pms, media_ops = [0.2], [0.0, 0.5, 1.0, 1.5], [0.0, 0.1, 0.2, 0.3, 0.4, 0.5], [[0.0], [0.5], [0.05, 0.95], [0.05, 0.5, 0.95]]

In [ ]:
max_iterations = 100

In [ ]:
aggregate_metrics = dict()
i=0
for epsilon in epsilons:
    for gamma in gammas:
        for pm in pms:
            for media_op in media_ops:
                respath = f'../res/'
                name = f'e{epsilon}_g{gamma}_pm{pm}_mo{media_op}_maxit{max_iterations}'
                final_opinions, final_niter = import_dicts(respath, name)

                metrics = am.aggregateMetrics(final_opinions, final_niter)
                metrics.compute_metrics()

                aggregate_metrics[i] = {
                        'epsilon': epsilon,
                        'gamma': gamma,
                        'pm': pm,
                        'media_op': media_op,
                        'avg_nc': metrics.get_avg_nc(),
                        'avg_entr': metrics.get_avg_entr(),
                        'avg_pwdist': metrics.get_avg_pwdist(),
                        'avg_op': metrics.get_avg_opinion(),
                        'avg_nit': metrics.get_avg_nit()
                    }
                i+=1

with open(f'../aggregate/aggregate_metrics.json', 'w') as ofile:
    json.dump(aggregate_metrics, ofile)

In [ ]:
with open(f'../aggregate/aggregate_metrics.json', 'r') as ifile:
    aggregate_metrics = json.load(ifile)

In [ ]:
import pandas as pd
import json

# Convert the data into a list of dictionaries
rows = []
for key, values in aggregate_metrics.items():
    row = {'index': int(key)}
    for sub_key, sub_values in values.items():
        if sub_key in ['epsilon', 'gamma', 'pm', 'media_op']:
            row[sub_key] = sub_values
        else:
            row[sub_key] = sub_values[0]
            row['std_' + sub_key] = sub_values[1]
    rows.append(row)

# Create the DataFrame
df = pd.DataFrame(rows)
df = df.drop('index', axis=1)

In [ ]:
df.to_csv('../aggregate/results.csv', index=False)

# Aggregate Plots

In [ ]:
df = pd.read_csv('../aggregate/results.csv')

In [ ]:
def hex_to_rgb(value):
    '''
    Converts hex to rgb colours
    value: string of 6 characters representing a hex colour.
    Returns: list length 3 of RGB values'''
    value = value.strip("#") # removes hash symbol if present
    lv = len(value)
    return tuple(int(value[i:i + lv // 3], 16) for i in range(0, lv, lv // 3))


def rgb_to_dec(value):
    '''
    Converts rgb to decimal colours (i.e. divides each value by 256)
    value: list (length 3) of RGB values
    Returns: list (length 3) of decimal values'''
    return [v/256 for v in value]

def get_continuous_cmap(hex_list, float_list=None):
    ''' creates and returns a color map that can be used in heat map figures.
        If float_list is not provided, colour map graduates linearly between each color in hex_list.
        If float_list is provided, each color in hex_list is mapped to the respective location in float_list. 
        
        Parameters
        ----------
        hex_list: list of hex code strings
        float_list: list of floats between 0 and 1, same length as hex_list. Must start with 0 and end with 1.
        
        Returns
        ----------
        colour map'''
    rgb_list = [rgb_to_dec(hex_to_rgb(i)) for i in hex_list]
    if float_list:
        pass
    else:
        float_list = list(np.linspace(0,1,len(rgb_list)))
    cdict = dict()
    for num, col in enumerate(['red', 'green', 'blue']):
        col_list = [[float_list[i], rgb_list[i][num], rgb_list[i][num]] for i in range(len(float_list))]
        cdict[col] = col_list
    cmp = LinearSegmentedColormap('my_cmp', segmentdata=cdict, N=256)
    return cmp

heatmap_hex_list = ['#357db0', '#ce2626']
heatmap_cmap=get_continuous_cmap(heatmap_hex_list)

In [ ]:
balanced = df[df['media_op']=='[0.05, 0.5, 0.95]']
balanced = balanced.rename(columns={'epsilon':'eps', 'gamma':'gam', 'pm':'p_media'})


In [ ]:
balanced

In [ ]:
polarised = df[df['media_op']=='[0.05, 0.95]']
polarised = polarised.rename(columns={'epsilon':'eps', 'gamma':'gam', 'pm':'p_media'})


In [ ]:
extremist = df[df['media_op']=='[0.0]']
extremist = extremist.rename(columns={'epsilon':'eps', 'gamma':'gam', 'pm':'p_media'})


In [ ]:
moderate = df[df['media_op']=='[0.5]']
moderate = moderate.rename(columns={'epsilon':'eps', 'gamma':'gam', 'pm':'p_media'})


In [ ]:
averages = [c for c in df.columns if c.startswith('avg_')]
medians = [c for c in df.columns if c.startswith('med_')]

In [ ]:
import os 
for setting in ['balanced', 'polarised', 'extremist', 'moderate']:
    if not os.path.exists(f'../plots/{setting}'):
         os.makedirs(f'../plots/{setting}')
    for measure in averages:
        if not os.path.exists(f'../plots/{setting}/{measure}'):
            os.mkdir(f'../plots/{setting}/{measure}')
    for measure in medians:
        if not os.path.exists(f'../plots/{setting}/{measure}'):
                os.mkdir(f'../plots/{setting}/{measure}')

In [ ]:
sns.set(font_scale=2.0)
plt.rcParams['font.family']='serif'
plt.rcParams['font.weight']='bold'

In [ ]:
#balanced setting
setting = 'balanced'
df = balanced      
for measure in averages:
    for eps in sorted(df.eps.unique()):
        fig, ax = plt.subplots(figsize=(5, 5), dpi=600)
        x = "p_media"
        y = "gam"
        data = df[df["eps"]==eps]
        d = data.pivot(index=x, columns=y, values=measure)
        if max(df[measure]) > 100:
            a=sns.heatmap(d, cbar=False, cbar_kws = dict(use_gridspec=False, location="top"), fmt=".0f", annot=False, ax = ax, cmap=heatmap_cmap, vmin=min(df[measure]), vmax=max(df[measure]), annot_kws={
                'fontsize': 14,
                'fontweight': 'bold',
                'fontfamily': 'serif'
            })
        elif max(df[measure]) > 10:
            a=sns.heatmap(d, cbar=False, cbar_kws = dict(use_gridspec=False, location="top"), fmt=".1f", annot=True, ax = ax, cmap=heatmap_cmap, vmin=min(df[measure]), vmax=max(df[measure]), annot_kws={
                'fontsize': 14,
                'fontweight': 'bold',
                'fontfamily': 'serif'
            })
        else:
            a=sns.heatmap(d, cbar=False, cbar_kws = dict(use_gridspec=False, location="top"), fmt=".2f", annot=True, ax = ax, cmap=heatmap_cmap, vmin=min(df[measure]), vmax=max(df[measure]), annot_kws={
                'fontsize': 14,
                'fontweight': 'bold',
                'fontfamily': 'serif'
            })
        ax.set_xlabel(r'$\gamma$', fontsize=30, fontfamily='serif')
        ax.set_ylabel(r'$p_m$', fontsize=30, fontfamily='serif')
        plt.savefig(f"../plots/{setting}/{measure}/{setting}_{measure}_e{eps}_heatmap.png", bbox_inches="tight")
        plt.close()  

#polarised setting
setting = 'polarised'
df = polarised           
for measure in averages:
    for eps in sorted(df.eps.unique()):
        fig, ax = plt.subplots(figsize=(5, 5), dpi=600)
        x = "p_media"
        y = "gam"
        data = df[df["eps"]==eps]
        d = data.pivot(index=x, columns=y, values=measure)
        if max(df[measure]) > 100:
            a=sns.heatmap(d, cbar=False, cbar_kws = dict(use_gridspec=False, location="top"), fmt=".0f", annot=False, ax = ax, cmap=heatmap_cmap, vmin=min(df[measure]), vmax=max(df[measure]), annot_kws={
                'fontsize': 14,
                'fontweight': 'bold',
                'fontfamily': 'serif'
            })
        elif max(df[measure]) > 10:
            a=sns.heatmap(d, cbar=False, cbar_kws = dict(use_gridspec=False, location="top"), fmt=".1f", annot=True, ax = ax, cmap=heatmap_cmap, vmin=min(df[measure]), vmax=max(df[measure]), annot_kws={
                'fontsize': 14,
                'fontweight': 'bold',
                'fontfamily': 'serif'
            })
        else:
            a=sns.heatmap(d, cbar=False, cbar_kws = dict(use_gridspec=False, location="top"), fmt=".2f", annot=True, ax = ax, cmap=heatmap_cmap, vmin=min(df[measure]), vmax=max(df[measure]), annot_kws={
                'fontsize': 14,
                'fontweight': 'bold',
                'fontfamily': 'serif'
            })
        ax.set_xlabel(r'$\gamma$', fontsize=30, fontfamily='serif')
        ax.set_ylabel(r'$p_m$', fontsize=30, fontfamily='serif')
        plt.savefig(f"../plots/{setting}/{measure}/{setting}_{measure}_e{eps}_heatmap.png", bbox_inches="tight")
        plt.close()
print('done')      

#extremist setting
setting = 'extremist'
df = extremist          
for measure in averages:
    for eps in sorted(df.eps.unique()):
        fig, ax = plt.subplots(figsize=(5, 5), dpi=600)
        x = "p_media"
        y = "gam"
        data = df[df["eps"]==eps]
        d = data.pivot(index=x, columns=y, values=measure)
        if max(df[measure]) > 100:
            a=sns.heatmap(d, cbar=False, cbar_kws = dict(use_gridspec=False, location="top"), fmt=".0f", annot=False, ax = ax, cmap=heatmap_cmap, vmin=min(df[measure]), vmax=max(df[measure]), annot_kws={
                'fontsize': 14,
                'fontweight': 'bold',
                'fontfamily': 'serif'
            })
        elif max(df[measure]) > 10:
            a=sns.heatmap(d, cbar=False, cbar_kws = dict(use_gridspec=False, location="top"), fmt=".1f", annot=True, ax = ax, cmap=heatmap_cmap, vmin=min(df[measure]), vmax=max(df[measure]), annot_kws={
                'fontsize': 14,
                'fontweight': 'bold',
                'fontfamily': 'serif'
            })
        else:
            a=sns.heatmap(d, cbar=False, cbar_kws = dict(use_gridspec=False, location="top"), fmt=".2f", annot=True, ax = ax, cmap=heatmap_cmap, vmin=min(df[measure]), vmax=max(df[measure]), annot_kws={
                'fontsize': 14,
                'fontweight': 'bold',
                'fontfamily': 'serif'
            })
        ax.set_xlabel(r'$\gamma$', fontsize=30, fontfamily='serif')
        ax.set_ylabel(r'$p_m$', fontsize=30, fontfamily='serif')
        plt.savefig(f"../plots/{setting}/{measure}/{setting}_{measure}_e{eps}_heatmap.png", bbox_inches="tight")
        plt.close()
print('done')

#moderate setting
setting = 'moderate'
df = moderate          
for measure in averages:
    for eps in sorted(df.eps.unique()):
        fig, ax = plt.subplots(figsize=(5, 5), dpi=600)
        x = "p_media"
        y = "gam"
        data = df[df["eps"]==eps]
        d = data.pivot(index=x, columns=y, values=measure)
        if max(df[measure]) > 100:
            a=sns.heatmap(d, cbar=False, cbar_kws = dict(use_gridspec=False, location="top"), fmt=".0f", annot=False, ax = ax, cmap=heatmap_cmap, vmin=min(df[measure]), vmax=max(df[measure]), annot_kws={
                'fontsize': 14,
                'fontweight': 'bold',
                'fontfamily': 'serif'
            })
        elif max(df[measure]) > 10:
            a=sns.heatmap(d, cbar=False, cbar_kws = dict(use_gridspec=False, location="top"), fmt=".1f", annot=True, ax = ax, cmap=heatmap_cmap, vmin=min(df[measure]), vmax=max(df[measure]), annot_kws={
                'fontsize': 14,
                'fontweight': 'bold',
                'fontfamily': 'serif'
            })
        else:
            a=sns.heatmap(d, cbar=False, cbar_kws = dict(use_gridspec=False, location="top"), fmt=".2f", annot=True, ax = ax, cmap=heatmap_cmap, vmin=min(df[measure]), vmax=max(df[measure]), annot_kws={
                'fontsize': 14,
                'fontweight': 'bold',
                'fontfamily': 'serif'
            })
        ax.set_xlabel(r'$\gamma$', fontsize=30, fontfamily='serif')
        ax.set_ylabel(r'$p_m$', fontsize=30, fontfamily='serif')
        plt.savefig(f"../plots/{setting}/{measure}/{setting}_{measure}_e{eps}_heatmap.png", bbox_inches="tight")
        plt.close()
print('done')